<a href="https://colab.research.google.com/github/jackie-Gung/Colab_ESAA/blob/main/2022_11_18_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Chapter 9. 추천 시스템**
---

### **08. 파이썬 추천 시스템 패키지 - Surprise**
---

#### **1. Surprise 패키지 소개**
- 앞에서 다루었던 콘텐츠/아이템/잠재요인 기반 협업 필터링 코드는 최적화나 수행 속도면에서 보완이 필요함
- 추천 시스템은 상업적으로 가치가 크기 때문에 별도의 패키지로 제공되면 활용도가 매우 높을 것으로 예상됨
- 대표적인 추천 시스템 구축을 위한 전용 패키지
- **Surprise 패키지 장점**
  - 다양한 **추천 알고리즘**을 적용 가능함
  - 사이킷런의 핵심 API와 유사하게 작성되어 **학습/테스트 데이터 분리, cross validation, GridSearchCV** 등 다양한 기능 제공함

In [11]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### **2. Surprise를 이용한 추천 시스템 구축**

In [12]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
# 무비렌즈 링크 오류..-> 사이트 에러 뜸
data = Dataset.load_builtin('ml-100k')

In [ ]:
trainset, testset = train_test_split(data,test_size=0.25,random_state=0)

In [ ]:
# SVD로 잠재 요인 협업 필터링 수행하기
algo = SVD()
algo.fit(trainset)

- **Surprise에서 추천을 예측하는 메서드**
  - **test():** 사용자-아이템 평점 데이터 세트 전체에 대해 추천 예측
  - **predict():** 개별 사용자와 영화에 대한 추천 평점 반환

##### **1) test()**

In [ ]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ',len(predictions))
print('prediction 결과 처음 5개 추출')
predictions[:5]

- **분석 결과**
  - 호출 결과: 파이썬 list, 크기는 입력 인자 데이터세트 크기와 동일

In [ ]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

##### **2) predict()**

In [ ]:
# 사용자/아이템 아이디는 문자열로 입력하기
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

##### **3) 추천 예측 평점과 실제 평점과의 차이**

In [ ]:
accuracy.rmse(predictions)

#### **3. Surprise 주요 모듈 소개**

##### **1) Dataset**
    1. Surprise는 데이터가 모두 로우 레벨로 된 세트만 적용 가능함
    2. 로우 레벨이 아닌 데이터는 로딩에서 제외됨
    3. 꼭 무비렌즈에서 다운받은 데이터가 아니라도 판디스 dataframe에서도 로딩할 수 있음
       * 칼럼순서가 반드시 사용자 아이디->아이템 아이디->평점 순이어야 함

##### **2) OS 파일 데이터를 Surprise 데이터 세트로 로딩**
- 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안됨

In [15]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('')
ratings.to_csv('', index=False,header=False)

In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5))
data = Dataset.load_from_file('', reader=reader)

- **무비렌즈 데이터 형식이 아닌 OS 파일인 경우**
      1. line_format(string): 칼럼을 순서대로 나열.입력된 문자열을 공백으로 분리해 칼럼으로 인식함
      2. sep(char): 칼럼을 분리하는 분리자. 디폴트는 '\t'. 판다스에서 입력받는 경우엔 기재할 필요x
      3. rating_scale(tuple,optional): 평점 값의 min~max 설정함. 디폴트는 (1,5)

- **SVD 행렬 분해 기법 이용해 추천 예측해보기**

In [ ]:
trianset,testset = train_test_split(data,test_size=0.25,random_state=0)

In [ ]:
algo = SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

##### **3) 판다스 DataFrame에서 Surprise 데이터 세트롤 로딩**
- 데이터 세트 순서 꼭 지켜야함 (사용자->아이템->평점)

In [ ]:
ratings = pd.read_csv('')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trianset,testset = train_test_split(data,test_size=0.25,random_state=0)

algo = SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

#### **4. Surprise 추천 알고리즘 클래스**
    1. SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 알고리즘
       - 비용함수: 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization 적용
    2. KNNBasic: 최근접 이웃 협업 필터링을 위한 알고리즘
    3. BaselineOnly: 사용자/아이템 bias를 감안한 SGD 베이스라인 알고리즘

- **SVD 입력 파라미터**
  - **n_factors:** 잠재 요인 k의 개수로 디폴트는 100. 커질수록 좋지만 과적합 문제 발생
  - **n_epochs:** SGD(Stochastic Gradient Descent)수행 시 반복 횟수로 디폴트는 20
  - **biased(bool):** 베이스라인 사용시 편향 적용 여부로 디폴트는 True

#### **5. 베이스라인 평점**
    1. 개념: 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것
    2. 전체 평균 평점+사용자 편향 점수+아이템 편향 점수 
       1) 전체 평균 평점: 모든 사용자의 아이템에 대한 평점을 평균한 값
       2) 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
       3) 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점


#### **6. 교차 검증과 하이퍼 파라미터 튜닝**

In [16]:
from surprise.model_selection import cross_validate

In [ ]:
ratings = pd.read_csv('')
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],v=5,verbose=True)

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20,40,60], 'n_factors':[50,100,200]}
gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

#### **7. Surprise를 이용한 개인화 영화 추천 시스템 구축**

- **사용자가 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천해보기**

In [ ]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
algo = SVD(n_factors=50,random_state=0)
algo.fit(data)
# train/test 분류하지 않았기 때문에 오류남

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(rating_file='',reader=reader)
trainset = data_folds.full_trainset() # 전체 데이터를 학습 데이터로 만들기

algo = SVD(n_epochs=20,n_factors=50,random_state=0)
algo.fit(trainset)

In [ ]:
movies = pd.read_csv('')
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

In [ ]:
uid=str(9)
iid=str(42)
pred = algo.predict(uid,iid,verbose=True)

In [ ]:
def get_unseen_surprise(ratings,movies,userId):
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
  total_movies= movies['movieId'].tolist()
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:',len(unseen_movies),
        '전체 영화 수:',len(total_movies))
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings,movies,9)

In [ ]:
def recomm_movie_by_surprise(algo,userId,unseen_movies,top_n=10):
  predictions = [algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]
  
  def sortkey_est(pred):
    return pred.est
  
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = v[:top_n]

  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id,title,rating) for id,title,rating in zip(top_movie_ids,top_movie_rating, top_movie_titles)]

  return top_movie_preds

In [ ]:
unseen_movies = get_unseen_surprise(ratings,movies,9)
top_movie_preds = recomm_movie_by_surprise(algo,9,unseen_movies,top_n=10)

print('#### Top 10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
  print(top_movie[1],':',top_movie[2])